# 파인튜닝(Fine-tuning) 가이드

---

## 1. 개요 및 환경 설정

### 1.1 파인튜닝이란?

- **파인튜닝(Fine-tuning)** 은 사전 학습된 모델을 특정 도메인이나 작업에 맞게 추가 학습시키는 과정

    - 도메인 특화 지식 습득 (예: ETF 투자 전문 지식)
    - 응답 스타일 및 포맷 커스터마이징
    - 일반 모델보다 높은 정확도
    - 비용 효율적 (처음부터 학습하는 것보다)

- **LLM vs 임베딩 모델**

    | 특징 | LLM 파인튜닝 | 임베딩 모델 파인튜닝 |
    |------|-------------|-------------------|
    | 목적 | 텍스트 생성, 질의응답 | 의미적 유사도 계산, 검색 |
    | 출력 | 자연어 텍스트 | 벡터 임베딩 |
    | 주요 사용처 | 챗봇, 요약, 번역 | RAG, 시맨틱 검색, 추천 |
    | 데이터 형식 | Q&A 쌍, 대화 | 유사 문장 쌍, triplets |


### 1.2 환경 설정

- **필수 라이브러리 설치**

    ```bash
    # Unsloth (LLM 파인튜닝용)
    pip install unsloth / uv pip install unsloth

    # Sentence Transformers (임베딩 모델용)
    pip install -U "sentence-transformers>=3.0"

    # 공통 라이브러리
    pip install datasets accelerate torch pandas
    pip install python-dotenv  # 환경변수 관리용
    ```

- **하드웨어 요구사항**

- **LLM 파인튜닝 (Unsloth)**
    - GPU: NVIDIA GPU 16GB+ VRAM 권장
    - 4-bit 양자화로 메모리 4배 절감 가능
    - Google Colab (무료 T4), Runpod, Lambda Labs 등 활용 가능

- **임베딩 모델 파인튜닝**
    - GPU: 8GB+ VRAM
    - CPU에서도 가능하지만 느림
    - Colab 무료 티어로도 충분

- **Hugging Face 토큰 설정**

In [ ]:
import os
from dotenv import load_dotenv
from getpass import getpass

load_dotenv()

# 또는 직접 입력
if "HUGGINGFACE_TOKEN" not in os.environ:
    os.environ["HUGGINGFACE_TOKEN"] = getpass("Hugging Face Token: ")
    
if "OPENAI_API_KEY" not in os.environ:
    use_openai = input("OpenAI API 사용? (y/n): ")
    if use_openai.lower() == 'y':
        os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key: ")

# Hugging Face 로그인
from huggingface_hub import login
login(token=os.environ["HUGGINGFACE_TOKEN"], add_to_git_credential=True)

print("✅ 환경 설정 완료!")

---

## 2. 데이터 수집 및 전처리

- 한국 ETF 시장 데이터를 LLM 파인튜닝에 적합한 형식으로 변환하는 과정을 학습
- 다양한 파인튜닝 방법론에 맞는 데이터셋을 구성하는 방법 처리

### 2.1 데이터 수집

In [ ]:
import pandas as pd

# CSV 파일에서 로드
df = pd.read_csv('./etf_info.csv', encoding='cp949')

# 데이터 확인
print(f"데이터 크기: {df.shape}")
print(f"컬럼: {df.columns.tolist()}")
df.head()

### 2.2 데이터 전처리

In [ ]:
# 한글 컬럼명 영문으로 매핑
column_mapping = {
    '표준코드': 'standard_code',
    '단축코드': 'ticker',
    '한글종목명': 'name_kr',
    '한글종목약명': 'short_name_kr',
    '영문종목명': 'name_en',
    '상장일': 'listing_date',
    '기초지수명': 'base_index',
    '지수산출기관': 'index_provider',
    '추적배수': 'tracking_multiplier',
    '복제방법': 'replication_method',
    '기초시장분류': 'base_market_category',
    '기초자산분류': 'base_asset_category',
    '상장좌수': 'listed_shares',
    '운용사': 'manager',
    'CU수량': 'cu_quantity',
    '총보수': 'total_expense_ratio',
    '과세유형': 'tax_type'
}

# 데이터프레임의 컬럼명 변경
df = df.rename(columns=column_mapping)

df.head()

### 2.3 Train/Test 분할

In [ ]:
from sklearn.model_selection import train_test_split

# 80/20 분할
train_df, test_df = train_test_split(
    df, 
    test_size=0.2, 
    random_state=1207,
    stratify=df['tax_type']  # 과세 유형별 균등 분할
)

print(f"Train: {len(train_df)} ({len(train_df)/len(df)*100:.1f}%)")
print(f"Test: {len(test_df)} ({len(test_df)/len(df)*100:.1f}%)")

# 저장
train_df.to_csv('./etf_train.csv', index=False)
test_df.to_csv('./etf_test.csv', index=False)

### 2.4 기본 데이터셋 생성

`(1) Alpaca 형식 (LLM용)`

In [ ]:
from datasets import Dataset

def create_alpaca_dataset(df):
    """기본 Alpaca 형식 데이터셋"""
    alpaca_data = []
    
    for _, row in df.iterrows():
        # 패턴 1: 기본 정보
        alpaca_data.append({
            "instruction": "다음 ETF의 기본 정보를 제공해주세요.",
            "input": f"{row['name_kr']} (종목코드: {row['ticker']})",
            "output": f"{row['name_kr']}은 {row['manager']}에서 운용하는 ETF입니다. "
                     f"기초지수는 {row['base_index']}이며, "
                     f"총보수는 연 {row['total_expense_ratio']:.2f}%입니다. "
                     f"{pd.to_datetime(row['listing_date']).strftime('%Y년 %m월')}에 상장되었습니다."
        })
        
        # 패턴 2: 투자 특징
        alpaca_data.append({
            "instruction": "이 ETF의 특징을 알려주세요.",
            "input": f"{row['name_kr']}",
            "output": f"{row['name_kr']}는 {row['base_index']}를 추종하는 ETF로, "
                     f"{row['replication_method']} 방식으로 운용됩니다. "
                     f"기초자산은 {row['base_asset_category']}이며, "
                     f"과세 유형은 {row['tax_type']}입니다."
        })
        
        # 패턴 3: 운용사 질문
        alpaca_data.append({
            "instruction": "이 ETF의 운용사 정보를 알려주세요.",
            "input": f"{row['name_kr']}",
            "output": f"{row['name_kr']}는 {row['manager']}에서 운용하고 있습니다. "
                     f"총보수는 연 {row['total_expense_ratio']:.2f}%입니다."
        })
        
        # 패턴 4: 투자 적합성 (자산 분류별)
        suitability = {
            '주식': '시장 상승기에 유리하며, 변동성이 높아 리스크 관리가 필요합니다.',
            '채권': '안정적인 수익을 추구하며, 금리 변동에 영향을 받습니다.',
            '원자재': '인플레이션 헤지 수단으로 활용 가능하며, 시장 변동성이 큽니다.',
        }
        
        advice = suitability.get(
            row['base_asset_category'], 
            '해당 자산군의 특성을 충분히 이해하고 투자하시기 바랍니다.'
        )
        
        alpaca_data.append({
            "instruction": "이 ETF는 어떤 투자자에게 적합한가요?",
            "input": f"{row['name_kr']}",
            "output": f"{row['name_kr']}는 {row['base_asset_category']} 자산에 투자하는 ETF입니다. "
                     f"{advice}"
        })
    
    return Dataset.from_list(alpaca_data)

# 데이터셋 생성
train_alpaca = create_alpaca_dataset(train_df)
test_alpaca = create_alpaca_dataset(test_df)

print(f"기본 Alpaca 데이터: Train {len(train_alpaca)}, Test {len(test_alpaca)}")

In [ ]:
train_alpaca

In [ ]:
test_alpaca

In [ ]:
train_alpaca[0]

In [ ]:
test_alpaca[0]

In [ ]:
# 데이터셋 저장
output_dir = "datasets"
os.makedirs(output_dir, exist_ok=True)

train_alpaca.save_to_disk(f"{output_dir}/train_alpaca")
test_alpaca.save_to_disk(f"{output_dir}/test_alpaca")

In [ ]:
from datasets import load_from_disk

# 저장된 데이터셋 불러오기
train_alpaca = load_from_disk(f"{output_dir}/train_alpaca")
test_alpaca = load_from_disk(f"{output_dir}/test_alpaca")

# 데이터셋 확인
print(f"Train: {len(train_alpaca)}")
print(f"Test: {len(test_alpaca)}")

In [ ]:
from datasets import DatasetDict

# Train과 Test를 분리해서 하나의 데이터셋으로 구성
llm_alpaca_dataset = DatasetDict({
    'train': train_alpaca,
    'test': test_alpaca
})

print(f"✅ 데이터셋 구성:")
print(f"   Train: {len(llm_alpaca_dataset['train'])}개")
print(f"   Test: {len(llm_alpaca_dataset['test'])}개")

# 사용 예시
print("\n학습용 샘플:")
print(llm_alpaca_dataset['train'][0])

print("\n평가용 샘플:")
print(llm_alpaca_dataset['test'][0])

In [ ]:
# 데이터셋 허깅페이스 업로드
llm_alpaca_dataset.push_to_hub(
    "******/etf-alpaca-llm-v1",  # 자신의 사용자명으로 변경
    private=True,  # 비공개 여부 선택
)

In [ ]:
# 데이터셋 허깅페이스 다운로드
from datasets import load_dataset

# 데이터셋 다운로드
train_alpaca = load_dataset("******/etf-alpaca-llm-v1", split="train")
test_alpaca = load_dataset("******/etf-alpaca-llm-v1", split="test")

# 데이터셋 확인
print(f"Train: {len(train_alpaca)}")
print(f"Test: {len(test_alpaca)}")


`(2) 임베딩용 Positive Pairs`

In [ ]:
def create_embedding_dataset(df):
    """Hard Negatives를 포함한 임베딩 데이터"""
    from itertools import combinations
    
    pairs = []
    
    # 1. 기본 Positive Pairs (위와 동일)
    for _, row in df.iterrows():
        pairs.append({
            "sentence1": f"{row['name_kr']}",
            "sentence2": f"{row['base_index']}를 추종하는 {row['manager']} 운용 ETF",
            "label": 1  # 유사함
        })
        
        pairs.append({
            "sentence1": f"종목코드 {row['ticker']}",
            "sentence2": f"{row['name_kr']} ETF",
            "label": 1
        })
    
    # 2. Hard Negatives: 같은 운용사지만 다른 ETF
    for manager in df['manager'].unique():
        manager_etfs = df[df['manager'] == manager]
        
        if len(manager_etfs) > 1:
            # 같은 운용사의 ETF 조합
            for (idx1, row1), (idx2, row2) in combinations(manager_etfs.iterrows(), 2):
                pairs.append({
                    "sentence1": f"{row1['name_kr']}",
                    "sentence2": f"{row2['name_kr']}",
                    "label": 0  # 유사하지 않음 (같은 운용사지만 다른 상품)
                })
    
    # 3. Hard Negatives: 같은 기초지수지만 다른 ETF
    for index in df['base_index'].unique():
        index_etfs = df[df['base_index'] == index]
        
        if len(index_etfs) > 1:
            for (idx1, row1), (idx2, row2) in combinations(index_etfs.iterrows(), 2):
                pairs.append({
                    "sentence1": f"종목코드 {row1['ticker']}",
                    "sentence2": f"{row2['name_kr']}",
                    "label": 0  # 다른 ETF
                })
    
    # 4. Easy Negatives: 완전히 다른 카테고리
    for _, row1 in df.iterrows():
        # 다른 자산군 선택
        diff_category = df[df['base_asset_category'] != row1['base_asset_category']]
        
        if len(diff_category) > 0:
            row2 = diff_category.sample(1).iloc[0]
            pairs.append({
                "sentence1": f"{row1['name_kr']}",
                "sentence2": f"{row2['name_kr']}",
                "label": 0
            })
    
    return Dataset.from_list(pairs)

# 데이터셋 생성
train_embedding = create_embedding_dataset(train_df)
test_embedding = create_embedding_dataset(test_df)

print(f"임베딩 데이터: Train {len(train_embedding)}, Test {len(test_embedding)}")

In [ ]:
train_embedding

In [ ]:
test_embedding

In [ ]:
train_embedding[0]

In [ ]:
test_embedding[-1]

In [ ]:
# 데이터셋 저장
output_dir = "datasets"
os.makedirs(output_dir, exist_ok=True)

train_embedding.save_to_disk(f"{output_dir}/train_embedding")
test_embedding.save_to_disk(f"{output_dir}/test_embedding")


In [ ]:
# 저장된 데이터셋 불러오기
train_embedding = load_from_disk(f"{output_dir}/train_embedding")
test_embedding = load_from_disk(f"{output_dir}/test_embedding")

# 데이터셋 확인
print(f"Train: {len(train_embedding)}")
print(f"Test: {len(test_embedding)}")

In [ ]:
from datasets import DatasetDict

# Train과 Test를 분리해서 하나의 데이터셋으로 구성
embedding_dataset = DatasetDict({
    'train': train_embedding,
    'test': test_embedding
})

# 데이터셋 허깅페이스 업로드
embedding_dataset.push_to_hub(
    "******/etf-embedding-v1",  # 자신의 사용자명으로 변경
    private=True,  # 비공개 여부 선택
)



In [ ]:
# 데이터셋 허깅페이스 다운로드
from datasets import load_dataset

# 데이터셋 다운로드
train_embedding = load_dataset("******/etf-embedding-v1", split="train")
test_embedding = load_dataset("******/etf-embedding-v1", split="test")

# 데이터셋 확인
print(f"Train: {len(train_embedding)}")
print(f"Test: {len(test_embedding)}")
